IMPORTS

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

import tensorflow as tf
from tensorflow.keras import layers, models

from sklearn.neural_network import BernoulliRBM
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

CODE TO LOAD MAX-MIN POOLED MFCC FILES 

In [23]:
def load_mfcc_files(directory):
    mfcc_data = []
    for file in os.listdir(directory):
            print(file)
            file_path = os.path.join(directory, file)
            data = pd.read_csv(file_path, header=None).values
            mfcc_data.append(data)
            print(data.shape)
    return mfcc_data

# Load MFCC data
asha = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Asha Bhosle")
bhavgeet = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Bhavgeet")
kishor = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Kishor Kumar")
lavni = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Lavni")
michael = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Michael Jackson")
national = load_mfcc_files("C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\National Anthem")

# Debug: Print the number of files loaded from each directory
print(f"Asha Bhosle: {len(asha)} files")
print(f"Bhavgeet: {len(bhavgeet)} files")
print(f"Kishor Kumar: {len(kishor)} files")
print(f"Lavni: {len(lavni)} files")
print(f"Michael Jackson: {len(michael)} files")
print(f"National Anthem: {len(national)} files")

Aaiye Meharban - Howrah Bridge 1958 128 Kbps_MFCC.csv
(20, 25000)
Aaja Aaja - Yeh Raaste Hain Pyaar Ke 128 Kbps_MFCC.csv
(20, 25000)
Aao Huzoor Tumko - Kismet 128 Kbps_MFCC.csv
(20, 25000)
Aasmaa - Saand Ki Aankh 128 Kbps_MFCC.csv
(20, 25000)
Aise Jalta Hai Jiya - 1920 128 Kbps_MFCC.csv
(20, 25000)
Allahu - Dev 128 Kbps_MFCC.csv
(20, 25000)
Andhere - 31st October 128 Kbps_MFCC.csv
(20, 25000)
Ankh Milaoongi - Fiza 128 Kbps_MFCC.csv
(20, 25000)
Baila Baila - Khwahish 128 Kbps_MFCC.csv
(20, 25000)
Bhanwara Bada Nadan Asha Bhosle - Sahib Bibi Aur Ghulam 128 Kbps_MFCC.csv
(20, 25000)
bollywood_MKS 1978 - O Saathi Re Tere Bina-Female_MFCC.csv
(20, 25000)
bollywood_MS 1966 - Jhumka Gira Re_MFCC.csv
(20, 25000)
bollywood_Rangeela 1995 - Tanha Tanha Yahan Pe_MFCC.csv
(20, 25000)
bollywood_STK 1982 - Kitne Bhi Tu(Female)_MFCC.csv
(20, 25000)
bollywood_UJ 1981 - Dil Cheez Kya Hai_MFCC.csv
(20, 25000)
bollywood_UJ 1981 - In Aankhon Ki Masti Ke_MFCC.csv
(20, 25000)
Chain Aapko Mila - Footpath 128 

COMBINING DATA INTO A SINGLE NUMPY ARRAY WITH LABELS

In [24]:
#repeat songs

asha = np.array(asha)
bhavgeet = np.array(bhavgeet)
kishor=np.array(kishor)
lavni=np.array(lavni)
michael=np.array(michael)
national=np.array(national)
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)

asha=asha[:-1]
bhavgeet = bhavgeet[:-1]
kishor=kishor[:-1]
lavni=lavni[:-3]

michael=michael[:-1]
print("new")
print(asha.shape)
print(bhavgeet.shape)
print(kishor.shape)
print(lavni.shape)
print(michael.shape)
print(national.shape)


(50, 20, 25000)
(50, 20, 25000)
(50, 20, 25000)
(52, 20, 25000)
(50, 20, 25000)
(49, 20, 25000)
new
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)
(49, 20, 25000)


In [25]:

# Combine data
data = np.concatenate((asha, bhavgeet, kishor, lavni, michael, national), axis=0)

# Create labels
labels = ['asha'] * len(asha) + ['bhavgeet'] * len(bhavgeet) + ['kishor'] * len(kishor) + ['lavni'] * len(lavni) + ['michael'] * len(michael) + ['national'] * len(national)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Combine data and labels into a single array
combined_data = np.empty((data.shape[0], data.shape[1], data.shape[2] + 1), dtype=object)
combined_data[:, :, :-1] = data
combined_data[:, 0, -1] = encoded_labels  # Add labels in the last column

# Check the shape of the combined data
print(f"Combined data shape: {combined_data.shape}")

#The shape of combined_data will be (294, 20, 25001), where the last element in the innermost dimension contains the encoded labels.

Combined data shape: (294, 20, 25001)


In [26]:
# Extract the data without the labels
X = combined_data[:, :, :-1]

# Extract the labels
y = combined_data[:, 0, -1]

# Print the shapes of X and y to verify
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
X = X.astype(np.float32)
y = y.astype(np.int32)


X shape: (294, 20, 25000)
y shape: (294,)


label encoding

In [27]:

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y)

# Print the encoded labels to verify
print(f"Encoded labels: {encoded_y}")

Encoded labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]


SPLITING TRAIN SET INTO TRAIN AND TEST

In [28]:
x_train, x_test, y_train, y_test= train_test_split(X, encoded_y,test_size=0.2)
print(x_train.shape)

(235, 20, 25000)


MODEL shit

In [ ]:
# Define the RBMLayer (assuming you have this implemented)
class RBMLayer(layers.Layer):
    def __init__(self, n_components):
        super(RBMLayer, self).__init__()
        self.n_components = n_components
        # Add your RBM layer initialization here

    def call(self, inputs):
        # Add your RBM layer logic here
        return inputs

def build_model(input_shape):
    model = models.Sequential()
    # Conv1D Layer: 64 filters, filter size of 5, stride of 1
    model.add(layers.Conv1D(64, 5, activation='relu', strides=1, input_shape=input_shape))
    # Pooling Layer: Pool size of 2
    model.add(layers.MaxPooling1D(pool_size=2))
    
    model.add(layers.Flatten())
    
    # RBM Layer 1: 100 hidden units
    rbm_layer1 = RBMLayer(n_components=100)
    model.add(rbm_layer1)
    
    # RBM Layer 2: 50 hidden units
    rbm_layer2 = RBMLayer(n_components=50)
    model.add(rbm_layer2)
    
    # Fully Connected Layer: 128 units
    model.add(layers.Dense(128, activation='relu'))
    
    # Output Layer: Softmax with 6 units for each class
    model.add(layers.Dense(6, activation='softmax'))
    
    return model

# Example usage
input_shape = (20, 25000)  # Adjusted to match the input data shape
model = build_model(input_shape)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Ensure x_train and y_train are NumPy arrays
x_train = np.array(x_train)
y_train = np.array(y_train)

# Check the shapes of x_train and y_train
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")

# Ensure x_train is a 3D NumPy array
# Train the model
model.fit(x_train, y_train, epochs=40, batch_size=32)

c:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


x_train shape: (235, 20, 25000)
y_train shape: (235,)
Epoch 1/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 5s 99ms/step - accuracy: 0.1723 - loss: 2870.6929
Epoch 2/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 134ms/step - accuracy: 0.4753 - loss: 1134.8612
Epoch 3/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 131ms/step - accuracy: 0.6932 - loss: 602.2224
Epoch 4/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 128ms/step - accuracy: 0.7165 - loss: 503.1475
Epoch 5/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.7523 - loss: 516.1505
Epoch 6/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.8832 - loss: 186.7500
Epoch 7/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 119ms/step - accuracy: 0.8922 - loss: 143.2679
Epoch 8/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.9622 - loss: 27.2201
Epoch 9/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - accuracy: 0.9620 - loss: 43.1266
Epoch 10/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - accuracy: 0.9628 - loss: 32.5761
Epoch 11/40
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 114ms/step - accuracy: 0.9470 - loss: 46.2008
E

model testing and summary

In [33]:
# Assuming X_test and y_test are your test data and labels
X_test = x_test.astype(np.float32)
y_test = y_test.astype(np.int32)  # Ensure y_test is integer type for sparse_categorical_crossentropy

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)

# Print the test accuracy
print(f"Test accuracy: {test_accuracy}")
model.summary()

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.7037 - loss: 655.8516
Test accuracy: 0.7118644118309021


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_6 (Conv1D)               │ (None, 16, 64)         │     8,000,064 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_6 (MaxPooling1D)  │ (None, 8, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_12 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rbm_layer_13 (RBMLayer)         │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,199,508 (92.31 MB)

 Trainable params: 8,066,502 (30.77 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 16,133,006 (61.54 MB)

In [34]:
# Define the function to extract features
def extract_features(model, layer_name, input_data):
    """
    Extract features from a specific layer of the model.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the layer to extract features from.
    input_data (numpy.ndarray): The input data.
    
    Returns:
    numpy.ndarray: The extracted features.
    """
    layer = model.get_layer(name=layer_name)
    feature_extractor = models.Model(inputs=model.input, outputs=layer.output)
    features = feature_extractor.predict(input_data)
    return features

# Example usage
layer_name = 'dense_12'  # Replace with the name of the layer you want to extract features from
X_train_features = extract_features(model, layer_name, x_train)
X_test_features = extract_features(model, layer_name, X_test)

# Print the shape of the extracted features
print("X_train_features shape:", X_train_features.shape)
print("X_test_features shape:", X_test_features.shape)

# Now you can use the extracted features for further processing, e.g., training a RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Train a RandomForestClassifier on the extracted features
clf = RandomForestClassifier(n_estimators=100)
clf.fit(X_train_features, y_train)

# Evaluate the classifier on the test features
test_accuracy = clf.score(X_test_features, y_test)
print(f"Test accuracy: {test_accuracy}")

AttributeError: The layer sequential_6 has never been called and thus has no defined input.

In [ ]:
import pandas as pd
import numpy as np
import glob
import os

def mean_max_pooling(data, target_size):
    current_size = data.shape[1]
    
    if current_size > target_size:
        # Downsampling by mean-max pooling
        factor = current_size / target_size
        pooled_data = []
        
        for i in range(target_size):
            start_idx = int(i * factor)
            end_idx = int((i + 1) * factor)
            segment = data.iloc[:, start_idx:end_idx]
            
            # Calculate mean and max for the segment
            mean_values = segment.mean(axis=1)
            max_values = segment.max(axis=1)
            # Combine mean and max to form the pooled result
            combined_values = (mean_values + max_values) / 2
            pooled_data.append(combined_values)
        
        data_resized = pd.DataFrame(pooled_data).T
    
    elif current_size < target_size:
        # Upsampling by padding
        padding = pd.DataFrame(0, index=data.index, columns=range(target_size - current_size))
        data_resized = pd.concat([data, padding], axis=1)
    else:
        # No resizing needed
        data_resized = data
    
    return data_resized

def resize_mfcc_files(input_dir, output_dir, target_size):
    # Get all CSV files in the input directory
    file_paths = glob.glob(os.path.join(input_dir, "*.csv"))
    if not file_paths:
        print("No CSV files found in the specified directory.")
        return
    
    # Process each file
    for file_path in file_paths:
        # Read the CSV file
        data = pd.read_csv(file_path, header=None)
        
        # Apply mean-max pooling to reach the target size
        data_resized = mean_max_pooling(data, target_size)
        print(data_resized.shape)
        # Save the resized data to the output directory
        file_name = os.path.basename(file_path)
        output_path = os.path.join(output_dir, file_name)
        data_resized.to_csv(output_path, index=False, header=False)
        print(f"Resized and saved: {output_path}")

# Parameters
input_dir = "C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train\\Lavni"   # Directory containing the original MFCC CSV files
output_dir = "C:\\Users\\kani1\\OneDrive\\Documents\\DS203 Programing for Data Science\\project\\mfcc_train_proccessed\\Lavni" # Directory to save the resized CSV files
target_size = 25000  # Set your chosen target column size here

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Resize all files to the target size
resize_mfcc_files(input_dir, output_dir, target_size)


(20, 25000)
Resized and saved: C:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\project\mfcc_train_proccessed\Lavni\Aase_Wajwa_Ki_Dholki_Siddharth_Jadhav_Manasi_Naik_Lavani_MandaliMP3_mfcc.csv
(20, 25000)
Resized and saved: C:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\project\mfcc_train_proccessed\Lavni\Aho_Sheth_Lay_Disan_Jhaliya_Bhet_Sheth_Lavani_Hiryachi_Angathi_Sonali_mfcc.csv
(20, 25000)
Resized and saved: C:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\project\mfcc_train_proccessed\Lavni\Amhi_Nahi_Ja_Lavani_Song_Ideachi_Kalpana_Marathi_Lavani_Songs_Swapnil_mfcc.csv
(20, 25000)
Resized and saved: C:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\project\mfcc_train_proccessed\Lavni\Apsara_Aali_Full_Song_Natarang_Sonalee_Kulkarni_Ajay_Atul_Marathi_mfcc.csv
(20, 25000)
Resized and saved: C:\Users\kani1\OneDrive\Documents\DS203 Programing for Data Science\project\mfcc_train_proccessed\Lavni\Ardhya_Ratrila_

code to visualize what cdbn is doing (subject to change)

In [ ]:
"""def visualize_feature_maps(model, layer_name, input_data):
    """
    Visualize the feature maps of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    input_data (numpy.ndarray): The input data.
    """
    layer = model.get_layer(name=layer_name)
    feature_map_model = models.Model(inputs=model.input, outputs=layer.output)
    feature_maps = feature_map_model.predict(input_data)
    
    n_features = feature_maps.shape[-1]
    fig, axes = plt.subplots(1, n_features, figsize=(20, 5))
    
    for i in range(n_features):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()

def visualize_filters(model, layer_name):
    """
    Visualize the filters of a convolutional layer.
    
    Parameters:
    model (tensorflow.keras.Model): The trained model.
    layer_name (str): The name of the convolutional layer.
    """
    layer = model.get_layer(name=layer_name)
    filters, biases = layer.get_weights()
    
    # Normalize filter values to 0-1 so we can visualize them
    f_min, f_max = filters.min(), filters.max()
    filters = (filters - f_min) / (f_max - f_min)
    
    n_filters = filters.shape[-1]
    fig, axes = plt.subplots(1, n_filters, figsize=(20, 5))
    
    for i in range(n_filters):
        f = filters[:, :, :, i]
        axes[i].imshow(f[:, :, 0], cmap='viridis')
        axes[i].axis('off')
    
    plt.show()"""